In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
import itertools as it
import time as time
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.ensemble import StackingRegressor

In [177]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#nan_cols = test.isna().sum()
#test1 = test.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)
#train1 = train.drop(nan_cols[nan_cols > 269].index.tolist(), axis = 1)

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# fit and transform the data
test_imputed = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)
train_imputed = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)

corr_matrix = train_imputed.corr().abs()

corr_y = train_imputed.corrwith(train_imputed['y'])
train_clean = train_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)
test_clean = test_imputed.drop(corr_y[corr_y.isna()].index.tolist(), axis = 1)


indices = np.where(np.triu(corr_matrix > 0.92, k=1))

high_corr = []
count = 0
for idx in range(len(indices[0])):
    col1 = corr_matrix.columns[indices[0][idx]]
    col2 = corr_matrix.columns[indices[1][idx]]
    high_corr.append(col1)
    count+=1

unique_vals = set(high_corr)
train_clean = train_clean.drop(unique_vals, axis = 1)

test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]

In [93]:
# DONT RUN
corr_matrix = train_clean.corr().abs()

corr_y = train_clean.corrwith(train_clean['y'])

low_corr_cols = corr_y[corr_y < 0.02].index.tolist() # 0.02 was best so far 

train_clean = train_clean.drop(low_corr_cols, axis = 1)
test_col = train_clean.drop('y', axis = 1)
test_clean = test_clean[test_col.columns]


In [178]:
X = train_clean.drop('y', axis = 1)
y = train_clean.y

X_test = test_clean

In [84]:
X_test.shape

(4403, 89)

In [179]:
# Top Predictors From MARS: # x102, x118, x355, x561, x253, x108, x591, x337, x420, x488
X['x102'] = train_imputed['x102']
X['x118'] = train_imputed['x118']
X['x561'] = train_imputed['x561']
X['x355'] = train_imputed['x355']
X['x585'] = train_imputed['x585']
X['x253'] = train_imputed['x253']
X['x108'] = train_imputed['x108']
X['x337'] = train_imputed['x337']
X['x420'] = train_imputed['x420']
X['x488'] = train_imputed['x488']


X_test['x102'] = test_imputed['x102']
X_test['x118'] = test_imputed['x118']
X_test['x561'] = test_imputed['x561']
X_test['x355'] = test_imputed['x355']
X_test['x585'] = test_imputed['x585']
X_test['x253'] = test_imputed['x253']
X_test['x108'] = test_imputed['x108']
X_test['x337'] = test_imputed['x337']
X_test['x420'] = test_imputed['x420']
X_test['x488'] = test_imputed['x488']

# Top Predictors from RF: x420, x355, x570, x561, x585, x105, x427, x353, x119, x102
X['x570'] = train_imputed['x570']
X['x427'] = train_imputed['x427']

X_test['x570'] = test_imputed['x570']
X_test['x427'] = test_imputed['x427']

In [185]:
# xgb_model = xgb.XGBRegressor(random_state=1,max_depth=6,n_estimators=1250,learning_rate=0.01,reg_lambda=0)

xgb_model = xgb.XGBRegressor(random_state=1,max_depth=6,n_estimators=500,learning_rate=0.01,reg_lambda=0.001)
xgb_model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1250, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [181]:
catboost = CatBoostRegressor(subsample=0.5,
                          reg_lambda=10, 
                          num_leaves=31, 
                          n_estimators=500, 
                          max_depth=6, 
                          learning_rate=0.05)
catboost.fit(X,y)

0:	learn: 11.9050906	total: 158ms	remaining: 1m 18s
1:	learn: 11.7881281	total: 290ms	remaining: 1m 12s
2:	learn: 11.6859812	total: 364ms	remaining: 1m
3:	learn: 11.5764956	total: 430ms	remaining: 53.3s
4:	learn: 11.4764682	total: 491ms	remaining: 48.6s
5:	learn: 11.3752380	total: 547ms	remaining: 45s
6:	learn: 11.2697950	total: 612ms	remaining: 43.1s
7:	learn: 11.1925674	total: 678ms	remaining: 41.7s
8:	learn: 11.1127020	total: 720ms	remaining: 39.3s
9:	learn: 11.0327554	total: 770ms	remaining: 37.7s
10:	learn: 10.9473513	total: 819ms	remaining: 36.4s
11:	learn: 10.8644619	total: 877ms	remaining: 35.7s
12:	learn: 10.8088679	total: 930ms	remaining: 34.8s
13:	learn: 10.7336550	total: 975ms	remaining: 33.9s
14:	learn: 10.6727045	total: 1.05s	remaining: 34.1s
15:	learn: 10.6139287	total: 1.13s	remaining: 34.1s
16:	learn: 10.5514224	total: 1.18s	remaining: 33.6s
17:	learn: 10.4834560	total: 1.23s	remaining: 32.9s
18:	learn: 10.4132085	total: 1.3s	remaining: 32.9s
19:	learn: 10.3464429	tota

163:	learn: 8.0648257	total: 10.5s	remaining: 21.5s
164:	learn: 8.0600338	total: 10.6s	remaining: 21.4s
165:	learn: 8.0559611	total: 10.6s	remaining: 21.4s
166:	learn: 8.0521918	total: 10.7s	remaining: 21.3s
167:	learn: 8.0384757	total: 10.7s	remaining: 21.2s
168:	learn: 8.0365330	total: 10.8s	remaining: 21.2s
169:	learn: 8.0289057	total: 10.9s	remaining: 21.1s
170:	learn: 8.0273137	total: 10.9s	remaining: 21.1s
171:	learn: 8.0243968	total: 11s	remaining: 21s
172:	learn: 8.0092364	total: 11.1s	remaining: 20.9s
173:	learn: 8.0061435	total: 11.1s	remaining: 20.8s
174:	learn: 8.0037757	total: 11.2s	remaining: 20.8s
175:	learn: 8.0017359	total: 11.2s	remaining: 20.7s
176:	learn: 7.9874139	total: 11.3s	remaining: 20.6s
177:	learn: 7.9851570	total: 11.3s	remaining: 20.5s
178:	learn: 7.9780460	total: 11.4s	remaining: 20.5s
179:	learn: 7.9656534	total: 11.5s	remaining: 20.4s
180:	learn: 7.9541307	total: 11.6s	remaining: 20.4s
181:	learn: 7.9494201	total: 11.6s	remaining: 20.3s
182:	learn: 7.94

323:	learn: 7.0303585	total: 21.6s	remaining: 11.7s
324:	learn: 7.0249329	total: 21.7s	remaining: 11.7s
325:	learn: 7.0156290	total: 21.7s	remaining: 11.6s
326:	learn: 7.0145293	total: 21.8s	remaining: 11.5s
327:	learn: 7.0122166	total: 21.8s	remaining: 11.4s
328:	learn: 7.0107177	total: 21.9s	remaining: 11.4s
329:	learn: 7.0062502	total: 21.9s	remaining: 11.3s
330:	learn: 7.0043025	total: 21.9s	remaining: 11.2s
331:	learn: 6.9913305	total: 22s	remaining: 11.1s
332:	learn: 6.9821131	total: 22.1s	remaining: 11.1s
333:	learn: 6.9716969	total: 22.1s	remaining: 11s
334:	learn: 6.9690852	total: 22.2s	remaining: 10.9s
335:	learn: 6.9580439	total: 22.2s	remaining: 10.8s
336:	learn: 6.9478773	total: 22.3s	remaining: 10.8s
337:	learn: 6.9440748	total: 22.3s	remaining: 10.7s
338:	learn: 6.9437088	total: 22.4s	remaining: 10.6s
339:	learn: 6.9418961	total: 22.4s	remaining: 10.6s
340:	learn: 6.9372523	total: 22.5s	remaining: 10.5s
341:	learn: 6.9289163	total: 22.5s	remaining: 10.4s
342:	learn: 6.92

486:	learn: 6.2673940	total: 30.2s	remaining: 806ms
487:	learn: 6.2621368	total: 30.2s	remaining: 744ms
488:	learn: 6.2555731	total: 30.3s	remaining: 681ms
489:	learn: 6.2492491	total: 30.3s	remaining: 619ms
490:	learn: 6.2478970	total: 30.4s	remaining: 556ms
491:	learn: 6.2476477	total: 30.4s	remaining: 494ms
492:	learn: 6.2420473	total: 30.5s	remaining: 432ms
493:	learn: 6.2366219	total: 30.5s	remaining: 370ms
494:	learn: 6.2290776	total: 30.5s	remaining: 308ms
495:	learn: 6.2237264	total: 30.6s	remaining: 247ms
496:	learn: 6.2169465	total: 30.6s	remaining: 185ms
497:	learn: 6.2119341	total: 30.7s	remaining: 123ms
498:	learn: 6.2056477	total: 30.7s	remaining: 61.5ms
499:	learn: 6.2053712	total: 30.8s	remaining: 0us


In [127]:
#Finding the lasso coefficients for increasing values of the tuning parameter
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

alphas = 10**np.linspace(10,-2,100)*0.1
lasso = Lasso(max_iter = 10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X,y)
    coefs.append(lasso.coef_)

alphas = 10**np.linspace(-1,-5,200)*0.5
lassocv = LassoCV(alphas = alphas, cv = 10, max_iter = 10000)
lassocv.fit(X,y)

#Optimal value of the tuning parameter - lamda
lassocv.alpha_

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.603e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.554e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.976e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.923e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.302e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.293e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 217317.72391846907, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 216992.16778441798, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 216668.80609719787, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anacon

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 210013.10279419084, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 209791.1769077828, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 209572.73389034928, tolerance: 69.62326850475013
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anacond

...

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200833.12135044276, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200802.14542107913, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200771.70022434925, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anacon

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200165.64468342686, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200146.24147844964, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 200127.3866145291, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anacond

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199825.4580132573, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199817.96988803407, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199810.78750037728, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anacond

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199705.6406649041, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199703.23252885204, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:617: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 199700.9303390358, tolerance: 69.76885983064845
  model = cd_fast.enet_coordinate_descent_gram(
/Users/ashleywitarsa/opt/anaconda

0.013063376127816646

In [182]:
lasso = Lasso(alpha = lassocv.alpha_)
lasso.fit(X,y)

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.358e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.013063376127816646)

In [186]:
en = StackingRegressor(estimators = [('lasso', lasso),('catboost', catboost),('xgb', xgb_model)],
                     final_estimator=LinearRegression(),                                          
                    cv = KFold(n_splits = 5, shuffle = True, random_state=1))
en.fit(X,y)

/Users/ashleywitarsa/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.358e+05, tolerance: 7.767e+01
  model = cd_fast.enet_coordinate_descent(


0:	learn: 11.9050906	total: 107ms	remaining: 53.5s
1:	learn: 11.7881281	total: 159ms	remaining: 39.6s
2:	learn: 11.6859812	total: 210ms	remaining: 34.9s
3:	learn: 11.5764956	total: 252ms	remaining: 31.3s
4:	learn: 11.4764682	total: 297ms	remaining: 29.4s
5:	learn: 11.3752380	total: 358ms	remaining: 29.4s
6:	learn: 11.2697950	total: 402ms	remaining: 28.3s
7:	learn: 11.1925674	total: 449ms	remaining: 27.6s
8:	learn: 11.1127020	total: 501ms	remaining: 27.3s
9:	learn: 11.0327554	total: 601ms	remaining: 29.5s
10:	learn: 10.9473513	total: 685ms	remaining: 30.4s
11:	learn: 10.8644619	total: 742ms	remaining: 30.2s
12:	learn: 10.8088679	total: 795ms	remaining: 29.8s
13:	learn: 10.7336550	total: 872ms	remaining: 30.3s
14:	learn: 10.6727045	total: 948ms	remaining: 30.7s
15:	learn: 10.6139287	total: 1.01s	remaining: 30.5s
16:	learn: 10.5514224	total: 1.07s	remaining: 30.4s
17:	learn: 10.4834560	total: 1.18s	remaining: 31.5s
18:	learn: 10.4132085	total: 1.23s	remaining: 31.2s
19:	learn: 10.3464429	

161:	learn: 8.0778497	total: 11.7s	remaining: 24.3s
162:	learn: 8.0731478	total: 11.7s	remaining: 24.2s
163:	learn: 8.0648257	total: 11.7s	remaining: 24s
164:	learn: 8.0600338	total: 11.8s	remaining: 23.9s
165:	learn: 8.0559611	total: 11.8s	remaining: 23.7s
166:	learn: 8.0521918	total: 11.8s	remaining: 23.6s
167:	learn: 8.0384757	total: 11.9s	remaining: 23.5s
168:	learn: 8.0365330	total: 11.9s	remaining: 23.3s
169:	learn: 8.0289057	total: 11.9s	remaining: 23.2s
170:	learn: 8.0273137	total: 12s	remaining: 23.1s
171:	learn: 8.0243968	total: 12s	remaining: 22.9s
172:	learn: 8.0092364	total: 12.1s	remaining: 22.8s
173:	learn: 8.0061435	total: 12.1s	remaining: 22.7s
174:	learn: 8.0037757	total: 12.1s	remaining: 22.5s
175:	learn: 8.0017359	total: 12.2s	remaining: 22.4s
176:	learn: 7.9874139	total: 12.2s	remaining: 22.3s
177:	learn: 7.9851570	total: 12.2s	remaining: 22.2s
178:	learn: 7.9780460	total: 12.3s	remaining: 22s
179:	learn: 7.9656534	total: 12.3s	remaining: 21.9s
180:	learn: 7.954130

325:	learn: 7.0156290	total: 19.1s	remaining: 10.2s
326:	learn: 7.0145293	total: 19.1s	remaining: 10.1s
327:	learn: 7.0122166	total: 19.2s	remaining: 10.1s
328:	learn: 7.0107177	total: 19.2s	remaining: 9.99s
329:	learn: 7.0062502	total: 19.3s	remaining: 9.92s
330:	learn: 7.0043025	total: 19.3s	remaining: 9.85s
331:	learn: 6.9913305	total: 19.3s	remaining: 9.78s
332:	learn: 6.9821131	total: 19.4s	remaining: 9.71s
333:	learn: 6.9716969	total: 19.4s	remaining: 9.66s
334:	learn: 6.9690852	total: 19.5s	remaining: 9.6s
335:	learn: 6.9580439	total: 19.5s	remaining: 9.53s
336:	learn: 6.9478773	total: 19.6s	remaining: 9.46s
337:	learn: 6.9440748	total: 19.6s	remaining: 9.4s
338:	learn: 6.9437088	total: 19.6s	remaining: 9.33s
339:	learn: 6.9418961	total: 19.7s	remaining: 9.26s
340:	learn: 6.9372523	total: 19.7s	remaining: 9.19s
341:	learn: 6.9289163	total: 19.8s	remaining: 9.13s
342:	learn: 6.9201086	total: 19.8s	remaining: 9.06s
343:	learn: 6.9078373	total: 19.8s	remaining: 8.99s
344:	learn: 6.

484:	learn: 6.2791337	total: 26.6s	remaining: 821ms
485:	learn: 6.2763226	total: 26.6s	remaining: 767ms
486:	learn: 6.2673940	total: 26.7s	remaining: 712ms
487:	learn: 6.2621368	total: 26.7s	remaining: 658ms
488:	learn: 6.2555731	total: 26.8s	remaining: 603ms
489:	learn: 6.2492491	total: 26.9s	remaining: 548ms
490:	learn: 6.2478970	total: 26.9s	remaining: 494ms
491:	learn: 6.2476477	total: 27.1s	remaining: 440ms
492:	learn: 6.2420473	total: 27.2s	remaining: 386ms
493:	learn: 6.2366219	total: 27.3s	remaining: 331ms
494:	learn: 6.2290776	total: 27.3s	remaining: 276ms
495:	learn: 6.2237264	total: 27.5s	remaining: 221ms
496:	learn: 6.2169465	total: 27.6s	remaining: 166ms
497:	learn: 6.2119341	total: 27.7s	remaining: 111ms
498:	learn: 6.2056477	total: 27.7s	remaining: 55.6ms
499:	learn: 6.2053712	total: 27.8s	remaining: 0us


...

435:	learn: 6.2999508	total: 18.6s	remaining: 2.73s
436:	learn: 6.2881565	total: 18.7s	remaining: 2.69s
437:	learn: 6.2808068	total: 18.7s	remaining: 2.65s
438:	learn: 6.2799709	total: 18.7s	remaining: 2.6s
439:	learn: 6.2791431	total: 18.8s	remaining: 2.56s
440:	learn: 6.2715842	total: 18.8s	remaining: 2.52s
441:	learn: 6.2647454	total: 18.8s	remaining: 2.47s
442:	learn: 6.2614529	total: 18.9s	remaining: 2.43s
443:	learn: 6.2575048	total: 18.9s	remaining: 2.39s
444:	learn: 6.2513522	total: 19s	remaining: 2.34s
445:	learn: 6.2407840	total: 19s	remaining: 2.3s
446:	learn: 6.2330852	total: 19.1s	remaining: 2.26s
447:	learn: 6.2256438	total: 19.1s	remaining: 2.22s
448:	learn: 6.2156459	total: 19.2s	remaining: 2.18s
449:	learn: 6.2054752	total: 19.2s	remaining: 2.13s
450:	learn: 6.2015484	total: 19.3s	remaining: 2.09s
451:	learn: 6.2005724	total: 19.3s	remaining: 2.05s
452:	learn: 6.1990409	total: 19.4s	remaining: 2.01s
453:	learn: 6.1975708	total: 19.4s	remaining: 1.97s
454:	learn: 6.1871

100:	learn: 8.5473988	total: 5.87s	remaining: 23.2s
101:	learn: 8.5374902	total: 5.91s	remaining: 23s
102:	learn: 8.5220622	total: 5.94s	remaining: 22.9s
103:	learn: 8.5204565	total: 5.98s	remaining: 22.8s
104:	learn: 8.5034686	total: 6.01s	remaining: 22.6s
105:	learn: 8.4892751	total: 6.05s	remaining: 22.5s
106:	learn: 8.4711339	total: 6.09s	remaining: 22.4s
107:	learn: 8.4633555	total: 6.13s	remaining: 22.3s
108:	learn: 8.4608764	total: 6.17s	remaining: 22.1s
109:	learn: 8.4591716	total: 6.2s	remaining: 22s
110:	learn: 8.4529766	total: 6.24s	remaining: 21.9s
111:	learn: 8.4407026	total: 6.27s	remaining: 21.7s
112:	learn: 8.4277712	total: 6.31s	remaining: 21.6s
113:	learn: 8.4257701	total: 6.35s	remaining: 21.5s
114:	learn: 8.4015101	total: 6.39s	remaining: 21.4s
115:	learn: 8.3923530	total: 6.44s	remaining: 21.3s
116:	learn: 8.3909436	total: 6.47s	remaining: 21.2s
117:	learn: 8.3800019	total: 6.51s	remaining: 21.1s
118:	learn: 8.3711858	total: 6.55s	remaining: 21s
119:	learn: 8.35724

264:	learn: 7.3173989	total: 12.3s	remaining: 10.9s
265:	learn: 7.3138358	total: 12.3s	remaining: 10.8s
266:	learn: 7.3076947	total: 12.4s	remaining: 10.8s
267:	learn: 7.2895355	total: 12.4s	remaining: 10.7s
268:	learn: 7.2867067	total: 12.4s	remaining: 10.7s
269:	learn: 7.2835637	total: 12.5s	remaining: 10.6s
270:	learn: 7.2751592	total: 12.5s	remaining: 10.6s
271:	learn: 7.2631536	total: 12.5s	remaining: 10.5s
272:	learn: 7.2625680	total: 12.6s	remaining: 10.5s
273:	learn: 7.2480216	total: 12.6s	remaining: 10.4s
274:	learn: 7.2399777	total: 12.7s	remaining: 10.4s
275:	learn: 7.2379499	total: 12.7s	remaining: 10.3s
276:	learn: 7.2215024	total: 12.7s	remaining: 10.3s
277:	learn: 7.2200860	total: 12.8s	remaining: 10.2s
278:	learn: 7.2156889	total: 12.8s	remaining: 10.2s
279:	learn: 7.2077527	total: 12.9s	remaining: 10.1s
280:	learn: 7.1891503	total: 12.9s	remaining: 10.1s
281:	learn: 7.1882604	total: 13s	remaining: 10s
282:	learn: 7.1665453	total: 13s	remaining: 9.97s
283:	learn: 7.1654

424:	learn: 6.4377807	total: 19.9s	remaining: 3.51s
425:	learn: 6.4254880	total: 20s	remaining: 3.47s
426:	learn: 6.4226386	total: 20.1s	remaining: 3.43s
427:	learn: 6.4183081	total: 20.1s	remaining: 3.38s
428:	learn: 6.4136673	total: 20.2s	remaining: 3.34s
429:	learn: 6.4016988	total: 20.3s	remaining: 3.3s
430:	learn: 6.3957987	total: 20.3s	remaining: 3.25s
431:	learn: 6.3839021	total: 20.4s	remaining: 3.21s
432:	learn: 6.3699961	total: 20.4s	remaining: 3.16s
433:	learn: 6.3685816	total: 20.5s	remaining: 3.11s
434:	learn: 6.3656909	total: 20.5s	remaining: 3.06s
435:	learn: 6.3605627	total: 20.6s	remaining: 3.02s
436:	learn: 6.3551134	total: 20.6s	remaining: 2.97s
437:	learn: 6.3444637	total: 20.7s	remaining: 2.93s
438:	learn: 6.3394391	total: 20.7s	remaining: 2.88s
439:	learn: 6.3385216	total: 20.8s	remaining: 2.83s
440:	learn: 6.3370982	total: 20.8s	remaining: 2.79s
441:	learn: 6.3244120	total: 20.9s	remaining: 2.74s
442:	learn: 6.3142027	total: 20.9s	remaining: 2.69s
443:	learn: 6.3

StackingRegressor(cv=KFold(n_splits=5, random_state=1, shuffle=True),
                  estimators=[('lasso', Lasso(alpha=0.013063376127816646)),
                              ('catboost',
                               <catboost.core.CatBoostRegressor object at 0x7fdfd048ff40>),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=1250, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=1, ...))],
                  final_estimator=LinearRegression())

In [187]:
pred = pd.DataFrame()
pred['id'] = test['id']
pred['y'] = en.predict(X_test)
pred = pred.set_index(['id'])
pred.to_csv('Witarsa_Ashley_regression.csv') # RMSE: 9.03624!!!